Follow the instructions here to set up a local instance of Nominatim!

https://www.linkedin.com/pulse/geocoding-geopy-your-own-nominatim-server-chonghua-yin/?trk=related_artice_Geocoding%20with%20GeoPy%20and%20Your%20Own%20Nominatim%20Server_article-card_title



In [1]:
from geopy.geocoders import Nominatim
geocoder = Nominatim(domain="localhost:8080", scheme="http")

In [2]:
geocoder.geocode("11710")

Location(Bellmore, Town of Hempstead, Nassau County, 11710, United States, (40.67664607465925, -73.53396529349835, 0.0))

In [3]:
import pandas as pd
zip_codes_df = pd.read_excel("../references/ZIP_Locale_Detail.xls", sheet_name=0)

We can filter on PHYSICAL STATE to get only the zip codes in NY. 

In [4]:
ny_zip_codes_df = zip_codes_df[zip_codes_df["PHYSICAL STATE"]=="NY"].copy()

Amusingly, there is one zip code that is physically in NY but serviced as part of Connecticut. This is Fisher's Island, which is in the sound but whose ferry service is from Connecticut. The more you know!

In [5]:
ny_zip_codes_df["DISTRICT NAME"].value_counts()

DISTRICT NAME
NEW YORK 3     1756
NEW YORK 2      309
NEW YORK 1      208
CONNECTICUT       1
Name: count, dtype: int64

We are interested primarily in Nassau and Suffolk counties. Happily, NYS has the boundaries of those counties available to us in Shapefile format. 

In [6]:
import geopandas as gpd
counties_gdf = gpd.read_file("../references/shapefiles/Counties.shp")

Also happily, the names are spelled the way we expect. 

In [7]:
long_island_gdf = counties_gdf[counties_gdf.NAME.isin(["Nassau","Suffolk"])].copy()
long_island_gdf = long_island_gdf.to_crs(4326)
long_island_gdf

,NAME,ABBREV,GNIS_ID,FIPS_CODE,SWIS,NYSP_ZONE,POP1990,POP2000,POP2010,POP2020,DOS_LL,DOSLL_DATE,NYC,CALC_SQ_MI,DATEMOD,Shape_Leng,Shape_Area,geometry
29,Nassau,NASS,974128,36059,280000,Long Island,1287348,1334544,1339532,1395774,NaN,NaN,N,446.637468,2018-04-12,168031.844843,1.156786e+09,"POLYGON ((-73.42898 40.97791, -73.42934 40.940..."
51,Suffolk,SUFF,974149,36103,470000,Long Island,1321864,1419369,1493350,1525920,NaN,NaN,N,2372.634185,NaN,385044.837960,6.145094e+09,"POLYGON ((-72.13717 40.90804, -72.15988 40.899..."


The zip code test! Now, we use the gecoder to get the latitude and longitude of the zip code (according to Nominatim). This will give us the set of points in NY. We will then perform a spatial join between these points and the dataframe containing the polygons for Nassau and Suffolk. 

In [8]:
ny_zip_codes_df[ny_zip_codes_df["PHYSICAL ZIP"]==11710]

,AREA NAME,AREA CODE,DISTRICT NAME,DISTRICT NO,DELIVERY ZIPCODE,LOCALE NAME,PHYSICAL DELV ADDR,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP,PHYSICAL ZIP 4
4290,ATLANTIC,4B,NEW YORK 2,117,11710,BELLMORE,2611 MERRICK RD,BELLMORE,NY,11710,5752.0
4291,ATLANTIC,4B,NEW YORK 2,117,11710,NORTH BELLMORE,2465 JERUSALEM AVE,NORTH BELLMORE,NY,11710,9991.0


Just for fun, I tested it with my own town. The location given appears to be the geometric center of Bellmore, rather than the address of one of its two post offices. That's certainly going to fall within Nassau county. 

In [9]:
result = geocoder.geocode("11710-9991", featuretype="postalcode", addressdetails=True)
result

Location(Bellmore, Town of Hempstead, Nassau County, 11710, United States, (40.67664607465925, -73.53396529349835, 0.0))

We don't get a different result if we use the zip+4 for the other address in Bellmore. This makes sense, as both places have the same zipcode effectively. 

In [10]:
ny_zip_codes_df

,AREA NAME,AREA CODE,DISTRICT NAME,DISTRICT NO,DELIVERY ZIPCODE,LOCALE NAME,PHYSICAL DELV ADDR,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP,PHYSICAL ZIP 4
2518,ATLANTIC,4B,CONNECTICUT,60,6390,FISHERS ISLAND,1 ORIENTAL AVE,FISHERS ISLAND,NY,6390,9992.0
3738,ATLANTIC,4B,NEW YORK 1,100,10001,GREELEY SQUARE,4 E 27TH ST,NEW YORK,NY,10001,9994.0
3739,ATLANTIC,4B,NEW YORK 1,100,10001,JAMES A FARLEY,421 8TH AVE,NEW YORK,NY,10001,9998.0
3740,ATLANTIC,4B,NEW YORK 1,100,10002,KNICKERBOCKER,128 E BROADWAY,NEW YORK,NY,10002,9998.0
3741,ATLANTIC,4B,NEW YORK 1,100,10003,COOPER,93 4TH AVE,NEW YORK,NY,10003,9998.0
...,...,...,...,...,...,...,...,...,...,...,...
6006,ATLANTIC,4B,NEW YORK 3,120,14903,ELMIRA HEIGHTS,209 E 14TH ST,ELMIRA,NY,14903,9993.0
6007,ATLANTIC,4B,NEW YORK 3,120,14904,ELMIRA,1580 SULLIVAN ST,ELMIRA,NY,14901,9998.0
6008,ATLANTIC,4B,NEW YORK 3,120,14904,SOUTH SIDE ELMIRA,418 S MAIN ST,ELMIRA,NY,14904,9993.0
6009,ATLANTIC,4B,NEW YORK 3,120,14905,DOWNTOWN ELMIRA,255 CLEMENS CENTER PKWY,ELMIRA,NY,14901,3090.0


In [11]:
ny_zip_only_df = ny_zip_codes_df[["PHYSICAL CITY", "LOCALE NAME", "DELIVERY ZIPCODE", "PHYSICAL ZIP"]].drop_duplicates()
ny_zip_only_df["postalcode"] = ny_zip_only_df["PHYSICAL ZIP"].apply(lambda x: str(int(x)).rjust(5, "0"))
ny_zip_only_df["geocode"] = ny_zip_only_df["postalcode"].apply(lambda x: geocoder.geocode(query={"postalcode": x}))

The surprising thing (perhaps) is that we are missing a fair amount of data. Taking one example, for Niobe NY there wasn't a zipcode associated with the data in Open Street Map (I've added one!). We can improve the data, but for our purposes it probably won't matter. There is enough interest in Long Island that we probably have pretty good coverage. 

In [12]:
ny_zip_only_df[ny_zip_only_df.geocode.isnull()]

,PHYSICAL CITY,LOCALE NAME,DELIVERY ZIPCODE,PHYSICAL ZIP,postalcode,geocode
3936,MAHOPAC FALLS,MAHOPAC FALLS,10542,10542,10542,None
3938,MARYKNOLL,MARYKNOLL,10545,10545,10545,None
3965,SHENOROCK,SHENOROCK,10587,10587,10587,None
4039,NEW MILFORD,NEW MILFORD,10959,10959,10959,None
4202,BROOKLYN,OZONE PARK CARRIER ANNEX,11416,11256,11256,None
...,...,...,...,...,...,...
5888,NIOBE,NIOBE,14758,14758,14758,None
5899,SAINT BONAVENTURE,SAINT BONAVENTURE,14778,14778,14778,None
5906,WEST CLARKSVILLE,WEST CLARKSVILLE,14786,14786,14786,None
5939,COOPERS PLAINS,COOPERS PLAINS,14827,14827,14827,None


We drop any records that were not geocoded successfully and apply a function to the rest to get a 2-dimensional point. 

In [13]:
from shapely.geometry import Point

ny_zip_geocoded_df = ny_zip_only_df.dropna().copy()
ny_zip_geocoded_df["Point"] = ny_zip_geocoded_df["geocode"].apply(lambda x: Point(x.longitude, x.latitude))

In [14]:
ny_zip_geocoded_df

,PHYSICAL CITY,LOCALE NAME,DELIVERY ZIPCODE,PHYSICAL ZIP,postalcode,geocode,Point
2518,FISHERS ISLAND,FISHERS ISLAND,6390,6390,06390,"(Fishers Island, Town of Southold, Suffolk Cou...",POINT (-72.00216730330368 41.267071742185514)
3738,NEW YORK,GREELEY SQUARE,10001,10001,10001,"(Manhattan, New York County, City of New York,...",POINT (-73.99403051640974 40.74839958433615)
3739,NEW YORK,JAMES A FARLEY,10001,10001,10001,"(Manhattan, New York County, City of New York,...",POINT (-73.99403051640974 40.74839958433615)
3740,NEW YORK,KNICKERBOCKER,10002,10002,10002,"(Manhattan, New York County, City of New York,...",POINT (-73.98921493881683 40.71714942163872)
3741,NEW YORK,COOPER,10003,10003,10003,"(Manhattan, New York County, City of New York,...",POINT (-73.98865170480549 40.731446913141546)
...,...,...,...,...,...,...,...
6006,ELMIRA,ELMIRA HEIGHTS,14903,14903,14903,"(Town of Horseheads, Chemung County, 14903, Un...",POINT (-76.84241289032967 42.12964614265934)
6007,ELMIRA,ELMIRA,14904,14901,14901,"(City of Elmira, Chemung County, 14901, United...",POINT (-76.80136979891394 42.09484067062178)
6008,ELMIRA,SOUTH SIDE ELMIRA,14904,14904,14904,"(City of Elmira, Chemung County, 14904, United...",POINT (-76.80386915599053 42.072885977696046)
6009,ELMIRA,DOWNTOWN ELMIRA,14905,14901,14901,"(City of Elmira, Chemung County, 14901, United...",POINT (-76.80136979891394 42.09484067062178)


In [15]:
ny_zip_geocoded_gdf = gpd.GeoDataFrame(ny_zip_geocoded_df, geometry=ny_zip_geocoded_df["Point"], crs=4326)

With our data in a dataframe, we can look at these zip code positions using Kepler.

In [16]:
from keplergl import KeplerGl
ny_map = KeplerGl(height=800)
ny_map.add_data(ny_zip_geocoded_gdf[["LOCALE NAME", "geometry"]])
ny_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [2518, 3738, 3739, 3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3…

We can now do a spatial join between the polygons representing Nassau and Suffolk counties, and the points (longitude and latitude) returned by our geocoder. 

In [17]:
long_island_zipcodes_df = gpd.sjoin(ny_zip_geocoded_gdf, long_island_gdf, predicate="within")

Bellmore only has one entry in the list, as we might expect. 

In [18]:
long_island_zipcodes_df[long_island_zipcodes_df["PHYSICAL CITY"]=='BELLMORE']

,PHYSICAL CITY,LOCALE NAME,DELIVERY ZIPCODE,PHYSICAL ZIP,postalcode,geocode,Point,geometry,index_right,NAME,...,POP2000,POP2010,POP2020,DOS_LL,DOSLL_DATE,NYC,CALC_SQ_MI,DATEMOD,Shape_Leng,Shape_Area
4290,BELLMORE,BELLMORE,11710,11710,11710,"(Bellmore, Town of Hempstead, Nassau County, 1...",POINT (-73.53396529349835 40.67664607465925),POINT (-73.53397 40.67665),29,Nassau,...,1334544,1339532,1395774,NaN,NaN,N,446.637468,2018-04-12,168031.844843,1.156786e+09


We can save our list of zipcodes to a file for use in filtering the Federal Election data later. 

In [19]:
long_island_zipcodes_df["address"] = long_island_zipcodes_df["geocode"].apply(lambda x: x.address)
long_island_zipcodes_df["locality"] = long_island_zipcodes_df["address"].apply(lambda x: x.split(",")[0])
long_island_zipcodes_df[["NAME", "address", "locality", "PHYSICAL ZIP"]].drop_duplicates().to_csv("../references/long_island_zipcodes.csv", index=None)